In [2]:
import pandas as pd
import configparser
import psycopg2
from io import StringIO
import tkinter as tk
from tkinter import messagebox, simpledialog

In [3]:
df = pd.read_csv('output_data/rushing_cleaned.csv')


In [4]:
# Get list of top QB 
get_qb = df[df['pos'] == 'QB']
get_qb.head(20)

player pos     yds    td
6      Justin Fields  QB  1143.0   8.0
32     Lamar Jackson  QB   764.0   3.0
33        Josh Allen  QB   762.0   7.0
35       Jalen Hurts  QB   760.0  13.0
37      Daniel Jones  QB   708.0   7.0
52    Marcus Mariota  QB   438.0   4.0
55      Kyler Murray  QB   418.0   3.0
60        Geno Smith  QB   366.0   1.0
61   Patrick Mahomes  QB   358.0   4.0
68   Trevor Lawrence  QB   291.0   5.0
73    Russell Wilson  QB   277.0   3.0
76        Joe Burrow  QB   257.0   5.0
78   Jacoby Brissett  QB   243.0   2.0
80     Kenny Pickett  QB   237.0   3.0
90      Dak Prescott  QB   182.0   1.0
91    Deshaun Watson  QB   175.0   1.0
96    Justin Herbert  QB   147.0   0.0
100    Tyler Huntley  QB   137.0   1.0
102     Malik Willis  QB   123.0   1.0
105      Davis Mills  QB   108.0   2.0

In [5]:
# Get list of top RB 
get_rb = df[df['pos'] == 'RB']
get_rb.head(20)

player pos     yds    td
0           Josh Jacobs  RB  1653.0  12.0
1         Derrick Henry  RB  1538.0  13.0
2            Nick Chubb  RB  1525.0  12.0
3        Saquon Barkley  RB  1312.0  10.0
4         Miles Sanders  RB  1269.0  11.0
5           Dalvin Cook  RB  1173.0   8.0
7   Christian McCaffrey  RB  1139.0   8.0
8        Travis Etienne  RB  1125.0   5.0
9           Aaron Jones  RB  1121.0   2.0
10      Jamaal Williams  RB  1066.0  17.0
11       Kenneth Walker  RB  1050.0   9.0
12  Rhamondre Stevenson  RB  1040.0   5.0
13       Tyler Allgeier  RB  1035.0   3.0
14         Najee Harris  RB  1034.0   7.0
15         Tony Pollard  RB  1007.0   9.0
16        Dameon Pierce  RB   939.0   4.0
17        Austin Ekeler  RB   915.0  13.0
18        DOnta Foreman  RB   914.0   5.0
19         Alvin Kamara  RB   897.0   2.0
20       Raheem Mostert  RB   891.0   3.0

In [6]:
# Get list of top WR 
get_wr = df[df['pos'] == 'WR']
get_wr.head(20)

player pos    yds   td
83           Deebo Samuel  WR  232.0  3.0
88          Curtis Samuel  WR  187.0  1.0
108           Velus Jones  WR  103.0  1.0
121      Amon-Ra St Brown  WR   95.0  0.0
124       Braxton Berrios  WR   91.0  2.0
130           Jamal Agnew  WR   86.0  0.0
133        Devin Duvernay  WR   84.0  1.0
134        Kadarius Toney  WR   82.0  1.0
137        Brandon Powell  WR   80.0  0.0
138      Christian Watson  WR   80.0  2.0
140       Ray-Ray McCloud  WR   78.0  1.0
149           Steven Sims  WR   70.0  0.0
153      Laviska Shenault  WR   65.0  1.0
159        Chase Claypool  WR   59.0  0.0
161       Parris Campbell  WR   58.0  0.0
162      Anthony Schwartz  WR   57.0  1.0
163        Rashid Shaheed  WR   57.0  1.0
165       Isaiah McKenzie  WR   55.0  1.0
170  Equanimeous St Brown  WR   54.0  0.0
172              DJ Moore  WR   53.0  0.0

In [7]:
# Get list of top QB 
get_te = df[df['pos'] == 'TE']
get_te.head(20)

player pos    yds   td
40         Taysom Hill  TE  575.0  7.0
211     Connor Heyward  TE   27.0  0.0
217      Armani Rogers  TE   26.0  0.0
250        Evan Engram  TE   13.0  0.0
262          Cole Kmet  TE    9.0  0.0
269       Mark Andrews  TE    8.0  0.0
274    Harrison Bryant  TE    8.0  0.0
293       Travis Kelce  TE    5.0  0.0
294        Jonnu Smith  TE    5.0  0.0
308      Tyler Conklin  TE    3.0  0.0
312  Chigoziem Okonkwo  TE    2.0  0.0
314   Daniel Bellinger  TE    2.0  1.0
318  Peyton Hendershot  TE    2.0  1.0
323      Durham Smythe  TE    1.0  1.0
325          Noah Gray  TE    1.0  1.0
334     Gerald Everett  TE    0.0  0.0
336     Feleipe Franks  TE    0.0  0.0
337      Kylen Granson  TE    0.0  0.0
362        David Njoku  TE   -8.0  0.0

In [8]:
# Import config and database 
config = configparser.ConfigParser()
config.read('db/config.ini')

dbname = config['database']['dbname']
user = config['database']['user']
password = config['database']['password']
host = config['database']['host']
port = config['database']['port']



In [17]:
# Execute the function to pick your starting line up use capital letters for first and last name 
# Make sure dbname, user, password, host, and port are defined before executing
class PickFantasyTeam:
    def __init__(self, root, dataframe):
        self.root = root
        self.root.title("Player Picker")
        self.df = dataframe
        self.picked_players = pd.DataFrame()
        self.unique_player_names = set()
        self.player_count = 0

        # Label and Entry for player name
        self.label_player_name = tk.Label(root, text="Enter player name (Use capital letters for first and last name ):")
        self.entry_player_name = tk.Entry(root)
        self.label_player_name.pack()
        self.entry_player_name.pack()

        # Button to add player
        self.add_player_button = tk.Button(root, text="Add Player", command=self.add_player)
        self.add_player_button.pack()

        # Text widget to display information
        self.text_display = tk.Text(root, height=10, width=40)
        self.text_display.pack()

        # Button to finish and push to Postgres
        self.finish_button = tk.Button(root, text="Finish and Push", command=self.finish_and_push)
        self.finish_button.pack()

    def add_player(self):
        # Set player name equal to input field
        player_name = self.entry_player_name.get()

        # Only allow 6 players on team
        if self.player_count > 5:
            messagebox.showwarning("Warning", "Only 6 players are allowed per team.")
            return
        # Make sure player name is entered
        if not player_name:
            messagebox.showwarning("Warning", "Player name cannot be empty.")
            return
        
        # Check to make sure there are no duplicate names
        if player_name in self.unique_player_names:
            messagebox.showwarning("Warning", f'{player_name} already exists in your team. Please enter a different name.')
            return

        player_data = self.get_players(player_name)

        # Attach added players to their DF data 
        if player_data is not None:
            self.picked_players = pd.concat([self.picked_players, player_data])
            self.unique_player_names.add(player_name)
            self.player_count += 1

            player_str = player_data.to_string(index=False)
            
            # Insert text into widge
            self.text_display.insert(tk.END, f'{player_str}\n')

            # Clear the entry widget
            self.entry_player_name.delete(0, tk.END)

    def get_players(self, player_name):
        # Check to make sure player exists in DF
        player_data = self.df[self.df['player'] == player_name]

        if not player_data.empty:
            return player_data
        else:
            # Show alert if player data can't be found 
            messagebox.showinfo("Information", f'No player found for {player_name}')
            return None

    def finish_and_push(self):
        # Team name input 
        team_name = simpledialog.askstring("Input", "Enter a team name (Do not use spaces):")

        # If field is empty send alert 
        if not team_name:
            messagebox.showwarning("Warning", "Team name cannot be empty.")
            return

        try:
            # Connect DB using psycopg2 (Enter your db information here before running)
            conn = psycopg2.connect(
                dbname=dbname,
                user=user,
                password=password,
                host=host,
                port=port
            )

            cursor = conn.cursor()

            # Create table in postgres 
            cursor.execute(f"""
                           CREATE TABLE IF NOT EXISTS {team_name}(
                            player VARCHAR(255),
                            pos VARCHAR(255),
                            yds FLOAT,
                            td FLOAT
                        );
                    """)

            # Copy and commit data to postgres 
            output = StringIO()
            self.picked_players.to_csv(output, sep='\t', header=False, index=False)
            output.seek(0)

            cursor.copy_from(output, f'{team_name.lower()}', null='', sep='\t')
            conn.commit()
            cursor.close()
            conn.close()

            messagebox.showinfo("Information", "Data pushed to Postgres.")

            # Output team to csv 
            self.picked_players.to_csv(f'output_data/{team_name}.csv', index=False)
            messagebox.showinfo("Information", f'Added {team_name} CSV to output_data directory.')

        except Exception as e:
            # Show any errors 
            messagebox.showerror("Error", f"An error occurred: {str(e)}")


if __name__ == "__main__":
    # Create Tkinter root window
    root = tk.Tk()

    # Create an instance of the PickFantasyTeam
    app = PickFantasyTeam(root, df)

    # Start the Tkinter main loop
    root.mainloop()
